# 2.5. 自动微分
<font size=4>
正如我们在 2.4节中所说的那样，求导是几乎所有深度学习优化算法的关键步骤。 虽然求导的计算很简单，只需要一些基本的微积分。 但对于复杂的模型，手工进行更新是一件很痛苦的事情（而且经常容易出错）。  <br><br>

深度学习框架通过自动计算导数，即自动微分（automatic differentiation）来加快求导。 实际中，根据我们设计的模型，系统会构建一个计算图（computational graph）， 来跟踪计算是哪些数据通过哪些操作组合起来产生输出。 自动微分使系统能够随后反向传播梯度。 这里，反向传播（backpropagate）意味着跟踪整个计算图，填充关于每个参数的偏导数。


</font>


## 2.5.1. 一个简单的例子
<font size=4>
    作为一个演示例子，假设我们想对函数$y=2\mathrm{x}^{\mathrm{T}}\mathrm{x}$关于列向量$\mathrm{x}$求导。 首先，我们创建变量$\mathrm{x}$并为其分配一个初始值。
</font>

In [55]:
import torch
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

<font size=4>
在我们计算$y$关于$\mathrm{x}$的梯度之前，我们需要一个地方来存储梯度。 重要的是，我们不会在每次对一个参数求导时都分配新的内存。 因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。 注意，一个标量函数$y$关于向量$\mathrm{x}$的梯度是向量，并且与具有相同的形状。
</font>

In [56]:
x.requires_grad_(True)  # 等价于x=torch.arange(4.0, requires_grad=True)
x.grad  # 默认情况下没有梯度

tensor([0., 1., 2., 3.], requires_grad=True)

<font size=4>
    现在让我们计算$y=2\mathrm{x}^{\mathrm{T}}\mathrm{x}$。
    </font>

In [57]:
y = 2 * torch.dot(x, x)  # 2*(0*0+1*1+2*2+3*3)=28
y

tensor(28., grad_fn=<MulBackward0>)

<font size=4>
    $\mathrm{x}$是一个长度为4的向量，计算$\mathrm{x}$和$\mathrm{x}$的点积，得到了我们赋值给$y$的标量输出。 接下来，我们通过调用反向传播函数来自动计算$y$关于向量$\mathrm{x}$的每个分量的梯度，并打印这些梯度。
</font>

In [58]:
y.backward()  # 反向传播，求标量y对向量x的每一个分量的梯度
x.grad  # y对向量x的每一个分量的梯度

tensor([ 0.,  4.,  8., 12.])

<font size=4>
    函数$y=2\mathrm{x}^{\mathrm{T}}\mathrm{x}$关于向量$\mathrm{x}$的梯度应为$4\mathrm{x}$。 让我们快速验证这个梯度是否计算正确。
</font>

In [59]:
x.grad == 4 * x

tensor([True, True, True, True])

<font size=4>
    现在让我们计算$\mathrm{x}$的另一个函数$y=x_{1}+x_{2}+\dots +x_{n}$。
</font>

In [60]:
x.grad.zero_()  # 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
y = x.sum()
y.backward()
x.grad

tensor([0., 0., 0., 0.])

tensor([1., 1., 1., 1.])

## 2.5.2. 非标量变量的反向传播
<font size=4>
当$\mathrm{y}$不是标量时，向量$\mathrm{y}$关于向量$\mathrm{x}$的导数的最自然解释是一个矩阵。 对于高阶和高维的$\mathrm{y}$和$\mathrm{x}$，求导的结果可以是一个高阶张量。<br><br>

然而，虽然这些更奇特的对象确实出现在高级机器学习中（包括深度学习中）， 但当我们调用向量的反向计算时，我们通常会试图计算损失函数对一批训练样本中的每个样本的组成部分的每个分量(特征)的导数。 这里，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。
</font>

In [61]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 在我们的例子中，我们只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()  # 清除x之前的梯度
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y
y.sum()
y.sum().backward()
x.grad

tensor([0., 0., 0., 0.])

tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)

tensor(14., grad_fn=<SumBackward0>)

tensor([0., 2., 4., 6.])

## 2.5.3. 分离计算
<font size=4>
    有时，我们希望将某些计算移动到记录的计算图之外。 例如，假设$y$是作为$\mathrm{x}$的函数计算的，而$z$则是作为$y$和$\mathrm{x}$的函数计算的。 想象一下，我们想计算$z$关于$\mathrm{x}$的梯度，但由于某种原因，我们希望将$y$视为一个常数， 并且只考虑到$\mathrm{x}$在$y$被计算后发挥的作用。

在这里，我们可以分离$y$来返回一个新变量$u$，该变量与$y$具有相同的值， 但丢弃计算图中如何计算$y$的任何信息。 换句话说，梯度不会向后流经$u$到$x$。 因此，下面的反向传播函数计算$z=u*\mathrm{x}$关于$\mathrm{x}$的偏导数，同时将$u$作为常数处理， 而不是$z=\mathrm{x}*\mathrm{x}*\mathrm{x}$关于$\mathrm{x}$的偏导数。
</font>

In [62]:
x.grad.zero_()  # 清除x之前的梯度
x  # 查看x
y = x * x   # 定义y
u = y.detach()  # detach()返回一个新的tensor，从当前计算图中分离下来。但是仍指向原变量的存放位置，不同之处只是requirse_grad为false.得到的这个tensir永远不需要计算器梯度，不具有grad.
z = u * x  # 此时u没有grad所以u被当做常数
z.sum().backward()
x.grad   # 由于u被看作常数所以z对x的梯度就为u
u == x.grad

tensor([0., 0., 0., 0.])

tensor([0., 1., 2., 3.], requires_grad=True)

tensor([0., 1., 4., 9.])

tensor([True, True, True, True])

<font size=4>
由于记录了$y$的计算结果，我们可以随后在$y$上调用反向传播， 得到$y=\mathrm{x}*\mathrm{x}$关于的$\mathrm{x}$的导数，即$2*\mathrm{x}$</font>

In [63]:
x.grad.zero_()  # 清除x之前的梯度
x
y.sum().backward()
x.grad
x.grad == 2*x

tensor([0., 0., 0., 0.])

tensor([0., 1., 2., 3.], requires_grad=True)

tensor([0., 2., 4., 6.])

tensor([True, True, True, True])

## 2.5.4. Python控制流的梯度计算
<font size=4>
    使用自动微分的一个好处是： 即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度。 在下面的代码中，while循环的迭代次数和if语句的结果都取决于输入a的值。
</font>

In [66]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

<font size=4>
    让我们计算梯度。
</font>

In [67]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

<font size=4>
我们现在可以分析上面定义的$f$函数。 请注意，它在其输入$a$中是分段线性的。 换言之，对于任何$a$，存在某个常量标量$k$，使得$f(a)=k*a$，其中$k$的值取决于输入$a$。 因此，我们可以用$d/a$验证梯度是否正确。
</font>

In [68]:
a.grad == d / a

tensor(True)

## 2.5.5. 小结
<font size=4>
深度学习框架可以自动计算导数：我们首先将梯度附加到想要对其计算偏导数的变量上。然后我们记录目标值的计算，执行它的反向传播函数，并访问得到的梯度。</font>

<font size=4>第五问里面，对f(x)进行求导，也就是非标量求导。是不是要计算sum()然后再backward,这里有点不太理解，非标量调用backward()函数要输入的gradient参数的具体意义。请问应该怎么理解？
<br><br>
如果y是矩阵，要先把y转化为标量，再求导。转化为方法是：backward()函数传入一个矩阵m，计算y*m（y的各元素与m的元素对应相乘，不是矩阵相乘），再求矩阵元素之和，这样得到一个标量（实际就是y中的元素加权求和），然后才能求导</font>